In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import imutils
import easyocr

import os

In [5]:
folder_path = 'data/images/one_car/0-99'
file_names = os.listdir(folder_path)

print(f"len(file_names) = {len(file_names)}")

len(file_names) = 91


In [ ]:
output_folder_path = 'data/plates/one_car/0-99/'

# Create the output folder if it doesn't exist
os.makedirs(output_folder_path, exist_ok=True)

license_plates_dict = {'image_name': [], 'x': [], 'y': [], 'width': [], 'height': [] }

for file_name in file_names:
    img = cv2.imread(os.path.join(folder_path, file_name))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Make the image gray
    
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) # Noise reduction
    edged = cv2.Canny(bfilter, 30, 200) # Edge detection
    
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # Find keypoints
    contours = imutils.grab_contours(keypoints) # Grab contours
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:30] # Select the 10 largest contours 
    
    # Find the position of the license plate (rectangle search)
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break
    
    if location is None:
        print("\nLOCATION IS NONE!!!")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(file_name)
        continue
        
    
    # Extract the license plate from the original image
    mask = np.zeros(gray.shape, np.uint8) # create a blank mask
    new_image = cv2.drawContours(mask, [location], 0, 255, -1) # draw contours inside the mask image with location coordinates
    new_image = cv2.bitwise_and(img, img, mask=mask) # Applying masks on top of the original image
    (x, y) = np.where(mask==255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2 + 1, y1:y2 + 1]
    
    
        
    (x, y, w, h) = cv2.boundingRect(location)
    
    license_plates_dict['image_name'].append(file_name)
    license_plates_dict['x'].append(x)
    license_plates_dict['y'].append(y)
    license_plates_dict['width'].append(w)
    license_plates_dict['height'].append(h)
    
    # Create a sign image using (x, y, w, h) information
    sign_image = img[y:y+h, x:x+w]
    
    output_file_path = os.path.join(output_folder_path, f'{file_name}_plate')
    cv2.imwrite(output_file_path, sign_image)
    
    # Create a figure with two subplots: one for the car image and one for the license plate image
    fig, axes = plt.subplots(1, 3, figsize=(12, 6))

    # Display the car image on the left subplot
    axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0].set_title(file_name)

    # Display the license plate image in the middle subplot
    axes[1].imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
    axes[1].set_title(file_name)
    
     # Display the sign image on the right subplot
    axes[2].imshow(cv2.cvtColor(sign_image, cv2.COLOR_BGR2RGB))
    axes[2].set_title('Sign Image')
    
    plt.show()  # Display the current image